# Student Loan Risk with Deep Learning

In [21]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [22]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [23]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [24]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [25]:
# Define the target set y using the credit_ranking column
y = loans_df["credit_ranking"]


# Display a sample of y
y[:5]


0    0
1    0
2    0
3    1
4    0
Name: credit_ranking, dtype: int64

In [26]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.copy()
X = X.drop("credit_ranking", axis=1)


# Review the features DataFrame
X.head()


,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


### Step 3: Split the features and target sets into training and testing datasets.


In [27]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [28]:
# Create a StandardScaler instance
scaler = StandardScaler()


# Fit the scaler to the features training dataset
scaler.fit(X_train)


# Fit the scaler to the features training dataset
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_scaled[:5]


array([[-0.56794375, -1.18088385, -0.19241296, -0.17192598, -0.45737585,
         1.86853316,  0.7479316 , -0.45120802,  0.73249574, -0.22553894,
        -0.31915735],
       [-0.00940983,  0.80065564, -0.04123135, -0.32236513, -0.0699402 ,
        -0.45384715, -0.08558383,  0.44779905, -0.03016065, -0.75844691,
        -1.15783431],
       [ 2.33643265, -0.84597577,  1.67216021,  0.05373275, -0.13451281,
         1.48146978,  0.80967349,  1.49315611, -1.36480934,  0.42579302,
         0.05358797],
       [ 0.21400374,  1.30301776, -0.24280683, -0.17192598, -0.50042426,
        -0.64737885, -0.11645477, -0.23168303, -1.36480934, -0.64002292,
        -0.97146166],
       [-0.00940983,  1.74956187, -1.30107808, -0.24714555, -0.37127904,
        -1.03444223, -0.98084114, -0.02261162,  0.03339405, -0.46238693,
         0.51951962]])

---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [29]:
# Define the the number of inputs (features) to the model
number_input_features = len(X_train.iloc[0])


# Review the number of features
number_input_features


11

In [30]:
# Define the number of hidden nodes for the first hidden layer
first_hidden_nodes = 8


# Define the number of hidden nodes for the second hidden layer
second_hidden_nodes = 5


# Define the number of neurons in the output layer
number_output_neurons = 1


In [31]:
# Create the Sequential model instance
nn = Sequential()


# Add the first hidden layer
nn.add(Dense(units=first_hidden_nodes, input_dim=number_input_features, activation="relu"))


# Add the second hidden layer
nn.add(Dense(units=second_hidden_nodes, activation="relu"))


# Add the output layer to the model specifying the number of output neurons and activation function
nn.add(Dense(units=number_output_neurons, activation="sigmoid"))


c:\Users\acrai\anaconda3\envs\dev\lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [32]:
# Display the Sequential model summary
nn.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 8)              │            96 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 5)              │            45 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │             6 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 147 (588.00 B)

 Trainable params: 147 (588.00 B)

 Non-trainable params: 0 (0.00 B)

### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [33]:
# Compile the Sequential model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [34]:
# Fit the model using 50 epochs and the training data
model = nn.fit(X_train_scaled, y_train, epochs=50)


Epoch 1/50


38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 696us/step - accuracy: 0.4931 - loss: 0.7051 
Epoch 2/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 693us/step - accuracy: 0.5950 - loss: 0.6657
Epoch 3/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 556us/step - accuracy: 0.5974 - loss: 0.6593
Epoch 4/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 521us/step - accuracy: 0.6277 - loss: 0.6369
Epoch 5/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 462us/step - accuracy: 0.6481 - loss: 0.6298
Epoch 6/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 517us/step - accuracy: 0.6711 - loss: 0.6094
Epoch 7/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 506us/step - accuracy: 0.7063 - loss: 0.5830
Epoch 8/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 565us/step - accuracy: 0.6973 - loss: 0.6013
Epoch 9/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 523us/step - accuracy: 0.7203 - loss: 0.5717
Epoch 10/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 487us/step - accuracy: 0.7220 - loss: 0.5656
Epoch 11/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 474us/step - accuracy: 0.7123 - loss: 0.5726
Epoch 12/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 598us/step - accurac

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [35]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)


# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


13/13 - 0s - 5ms/step - accuracy: 0.7300 - loss: 0.5220
Loss: 0.521958589553833, Accuracy: 0.7300000190734863


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [36]:
# Set the model's file path
file_path = Path("student_loan_model.keras")


# Export your model to a keras file
nn.save(file_path)


---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [37]:
# Set the model's file path
file_path = Path("student_loan_model.keras")


# Load the model to a new object
nn_imported = tf.keras.models.load_model(file_path)


### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [38]:
# Make predictions with the test data
y_pred = nn_imported.predict(X_test_scaled)


# Display a sample of the predictions
y_pred[:5]


13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


array([[0.47871926],
       [0.24270183],
       [0.8700206 ],
       [0.7088314 ],
       [0.99061155]], dtype=float32)

In [39]:
# Save the predictions array to a DataFrame and round the predictions to binary results
results = pd.DataFrame(y_pred, columns=["Predicted"])

#Convert the predicted values to binary
results["Predicted"] = results["Predicted"].apply(lambda x: 1 if x >= 0.5 else 0)

results



,Predicted
0,0
1,0
2,1
3,1
4,1
...,...
395,1
396,0
397,1
398,0


### Step 4: Display a classification report with the y test data and predictions

In [40]:
# Print the classification report with the y test data and predictions
print(classification_report(y_test, results["Predicted"]))



              precision    recall  f1-score   support

           0       0.69      0.76      0.73       188
           1       0.77      0.70      0.73       212

    accuracy                           0.73       400
   macro avg       0.73      0.73      0.73       400
weighted avg       0.73      0.73      0.73       400



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

**1.To build a recommendation system for student loan options, it would be essential to collect a comprehensive set of data points that reflect both the financial background and academic profile of students. This would include payment history, to understand previous loan repayment behaviors. GPA ranking and stem degree score, as indicators of academic performance and field of study, which could influence earning potential. Financial aid score and total loan score, to assess previous financial aid use and borrowing history, and credit ranking as a direct measure of creditworthiness. Collecting data on time to completion could also help predict future loan needs based on the duration of study. This data is relevant and necessary as it covers multiple aspects that affect a student’s ability to repay loans, such as their academic success, financial habits, and the potential for future earnings, thus allowing for a more personalized and effective loan recommendation.**


**2. Given the nature of the data selected for the recommendation system, a content-based filtering approach seems most suitable. This method recommends items based on the features of the products (in this case, student loan options) and a profile of the user's credit history, which can be constructed from the collected data points. For instance, a student with a high gpa ranking and stem degree score might be recommended different loan options compared to someone with a lower academic performance, based on the assumption of different earning potentials. Content-based filtering is suitable here because it allows for personalized recommendations based on specific attributes of the loans and the individual financial and academic characteristics of each student, rather than relying on the preferences of similar users (collaborative filtering) or contextual factors alone.**


**3. Data Privacy and Security: When collecting and using personal data such as credit scores, payment history, and financial aid information, a major challenge is ensuring the privacy and security of this sensitive information. This concern is critical because mishandling such data could lead to serious privacy breaches and financial fraud. Ensuring compliance with data protection regulations (like GDPR or CCPA) and implementing robust security measures would be essential.
Another significant challenge is avoiding bias in recommendations that could disadvantage certain groups of students. For example, if the model disproportionately recommends less favorable loan options to students from specific demographics or socioeconomic backgrounds, it could exacerbate existing inequalities. This challenge requires careful consideration of the data used and the algorithm's design to ensure that recommendations are fair and equitable for all students. Addressing these challenges is crucial for building a trustworthy and effective recommendation system that serves the diverse needs of students seeking loans.**

